In [111]:
import pandas as pd
import nltk
import math
import numpy

In [2]:
df = pd.DataFrame()

# Combining the 13 CSVs of CEO Tweets

In [5]:
import glob, os   #C:\Users\Bloody Dachi\Documents\CS_401\Final_Project\CEO-Topic-Classifier-Extractor\Finished_Tweet_CSV
path =r'C:/Users/Bloody Dachi/Documents/CS_401/Final_Project/CEO-Topic-Classifier-Extractor/Finished_Tweet_CSV' # use your path
allFiles = glob.glob(path + "/*.csv")

list_ = []

for file_ in allFiles: 
    df = pd.read_csv(file_,index_col=None, encoding = "latin1",lineterminator='\n')
    list_.append(df)
frame = pd.concat(list_, axis = 0, ignore_index = True)

In [6]:
#frame = frame.drop(['Unnamed: 0'], axis=1)
frame.head()

,Unnamed: 0,Tweet,CEO_Full_Name,CEO_User_Name
0,0,@pjournel The bag itself is from dsptch in San...,Phil Libin,plibin\r
1,1,I MacGyvered extra storage pouches to my backp...,Phil Libin,plibin\r
2,2,"Snack times Nuts equals Snuts, people. Itâs ...",Phil Libin,plibin\r
3,3,Looking forward to being back in Beijing this ...,Phil Libin,plibin\r
4,4,"For better and for worse, but mostly for the b...",Phil Libin,plibin\r


In [7]:
len(frame)

903277

## Creating Multi-Index to reference Tweets of specific CEOs

In [8]:
arrays = [frame["CEO_Full_Name"]]
index = pd.MultiIndex.from_arrays(arrays, names=['CEO_Names'])

In [9]:
df_new = pd.DataFrame(list(frame["Tweet"]), index=index, columns = ["Tweets"])
(df_new).head()

,Tweets
CEO_Names,
Phil Libin,@pjournel The bag itself is from dsptch in San...
Phil Libin,I MacGyvered extra storage pouches to my backp...
Phil Libin,"Snack times Nuts equals Snuts, people. Itâs ..."
Phil Libin,Looking forward to being back in Beijing this ...
Phil Libin,"For better and for worse, but mostly for the b..."


In [10]:
df_new.loc['Tom Hood']['Tweets'].values#.head()

array(['Time to rebalance? RT @AccountingEdit: Manuel: When it comes to the economy â\x80\x93 buckle your seatbelts! #dcpa18',
       'Agree! RT @mmaycpa: What a treat to have @AnjaManuel1 presenting at #DCPA18. Very informative presentation about tech, &amp; global players China, India.',
       'RT @AccountingEdit: Manuel: If thereâ\x80\x99s an AI race between the U.S. and China, itâ\x80\x99s not at all clear whoâ\x80\x99s going to come out on top. #dcpa18',
       ...,
       'RT @maragoni: #FutureReady is for everyone. Non-financial #blockchain uses will touch everything from real estate to app development to diamond industry: @ronqman #DCPA17',
       'Listening to @rmp289 of @ChristensenInst about how to think about disruption #dcpa17 https://t.co/XMO4WGHCT1',
       'He was great and he is coming to Maryland next Weds to open our CPA Summit - https://t.co/VrZsHNI5Uv live and by webcast. #futureready https://t.co/Rs1pSciR5o'],
      dtype=object)

# Perform LDA

In [11]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
token_words = []
for i in df_new.loc['Elon Musk']['Tweets'].values:
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    token_words.append(tokens)
    
    #tokens = tokenizer.tokenize(token_words)
print(token_words[0:3])

[['nbcnews', 'this', 'is', 'completely', 'backwards', 'based', 'on', 'what', 'we', 've', 'learned', 'from', 'the', 'hawthorne', 'test', 'tunnel', 'we', 're', 'moving', 'forward', 'with', 'a', 'much', 'larger', 'tunnel', 'network', 'under', 'la', 'won', 't', 'need', 'a', 'second', 'test', 'tunnel', 'under', 'sepulveda'], ['you', 'can', 'summon', 'your', 'tesla', 'from', 'your', 'phone', 'only', 'short', 'distances', 'today', 'but', 'in', 'a', 'few', 'years', 'summon', 'will', 'work', 'from', 'across', 'the', 'continent', 'https', 't', 'co', 'xcj67ajz8h'], ['cool', 'actually', 'if', 'you', 'buy', 'a', 'tesla', 'without', 'a', 'test', 'drive', 'you', 'have', '3', 'days', 'to', 'return', 'it', 'if', 'you', 'buy', 'after', 'a', 'test', 'drive', 'you', 'still', 'have', '24', 'hours', 'trying', 'to', 'incent', 'buying', 'with', 'no', 'test', 'drive', 'https', 't', 'co', 'o2dd5bgxrz']]


## Lemmzatizing and Stemming of Words

In [12]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    texts = [p_stemmer.stem(i) for i in stop_free]
    punc_free = ''.join(ch for ch in texts if ch not in exclude)
    
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in df_new.loc['Elon Musk']['Tweets'].values]

In [13]:
print(doc_clean[:3])

[['nbcnews', 'completely', 'backwards', 'based', 'we\x92ve', 'learned', 'hawthorne', 'test', 'tunnel', 'we\x92re', 'moving', 'forward', 'much', 'larger', 'tunnel', 'network', 'la', 'won\x92t', 'need', 'second', 'test', 'tunnel', 'sepulveda'], ['summon', 'tesla', 'phone', 'short', 'distance', 'today', 'year', 'summon', 'work', 'across', 'continent', 'httpstcoxcj67ajz8h'], ['cool', 'actually', 'buy', 'tesla', 'without', 'test', 'drive', '3', 'day', 'return', 'it', 'buy', 'test', 'drive', 'still', '24', 'hour', 'trying', 'incent', 'buying', 'test', 'drive', 'httpstcoo2dd5bgxrz']]


# TF-IDF

In [81]:
print(len(df_new.loc['Elon Musk']))

1791


In [187]:
# KEEP

# tweetDictionary = {}
# for i in df_new.index.unique():
#     tweetDictionary[i[0]] = df_new.loc[i[0]]['Tweets'].values
# for x in tweetDictionary:
#     print(x)

In [104]:
def tf_idf_matrix(dic):
    docs = dic
    n = len(docs)
    tf = {}
    df = {}
    length = {}
    vocab = set()
    for doc in docs:#list of names
        tweets = docs[doc]
        for i in range(len(tweets)):
            if type(tweets[i]) == str:
                words = nltk.word_tokenize(tweets[i])
                words = [p_stemmer.stem(i) for i in words if i not in stop] #remove stopwords
                words = [ch for ch in words if ch not in exclude]
                length[doc] = 0
                for word in words:
                    if not word.isalpha():
                        continue
                    length[doc] += 1
                    word = word.lower()
                    vocab.add(word)
                    if (word, doc) in tf:
                        tf[word, doc] += 1
                    else:
                        tf[word, doc] = 1
                    if word in df:
                        df[word].add(doc)
                    else:
                        df[word] = set([doc])

    tf_idf = {}
    for word, doc in tf:
        tf_idf[word, doc] = (tf[word, doc] / length[doc]) * math.log(n / len(df[word]), 10)
        #tf_idf[word, doc] = (1 + math.log(tf[word, doc], 10)) * math.log(n / len(df[word]), 10)
    return tf_idf, vocab, docs.keys()

def cos(v1, v2):
    if numpy.linalg.norm(v1) == 0 or numpy.linalg.norm(v2) == 0:
        return 0
    return numpy.dot(v1, v2) / (numpy.linalg.norm(v1) * numpy.linalg.norm(v2))


def getVector(word, tf_idf, docs):
    n = len(docs)
    v = numpy.zeros(n)

    for i in range(n):
        doc = docs[i]
        if (word, doc) in tf_idf:
            v[i] = tf_idf[word, doc]

    return v

In [93]:
tf_idf, vocab, docs = tf_idf_matrix(tweetDictionary)

## Selecting specific CEO to compare tweets against list of other CEOs. This will find the 30 closest CEOs who post similar tweets.

In [115]:
docs = list(docs)
name = "Elon Musk"
tweets = tweetDictionary[name]
total_words = []
for i in range(len(tweets)):
    if type(tweets[i]) == str:
        words = nltk.word_tokenize(tweets[i])
        words = [p_stemmer.stem(i) for i in words if i not in stop] #remove stopwords
        words = [ch for ch in words if ch not in exclude]
        for word in words:
            if word not in total_words:
                total_words.append(word)
sum_words = len(total_words)
sum_w_vec = numpy.zeros(len(docs))
for word in total_words:
    wordVector = getVector(word, tf_idf, docs)
    sum_w_vec = wordVector + sum_w_vec
doc_Cen = sum_w_vec/sum_words

cosines = {}
total_words = []
for doc in docs:#list of names
    tweets = tweetDictionary[doc]
    for i in range(len(tweets)):
        if type(tweets[i]) == str:
            words = nltk.word_tokenize(tweets[i])
            words = [p_stemmer.stem(i) for i in words if i not in stop] #remove stopwords
            words = [ch for ch in words if ch not in exclude]
            for word in words:
                if word not in total_words:
                    total_words.append(word)
    word_sum = len(total_words)
    doc_sum = numpy.zeros(len(docs))
    for WORD in total_words:
        v = getVector(WORD, tf_idf, docs)
        doc_sum = doc_sum + v

        centroid = doc_sum/word_sum
        cosines[doc] = cos(doc_Cen, centroid)

c2 = [(cosines[word], word) for word in cosines]
c2.sort()
print('\nCEOs most similar to ' + "''" + name + "''"+ ':')
for c, w in reversed(c2[-30:]):
    print('   {0:<12} {1:<.6f}'.format(w, c)) #prints out 30 most similar CEO's to Elon Musk
print()


CEOs most similar to ''Elon Musk'':
   Lauren Cook  0.970864
   Don Tapscott 0.964673
   Phil Libin   0.964533
   Sundar Pichai 0.963710
   Tom Hood     0.963100
   Joni Thomas Doolin 0.961163
   Jason Byrne  0.941851
   evÅk ad agency 0.941810
   RobLane      0.941674
   Richard Jalichandra 0.941648
   Bob Pritchett 0.941481
   Peter Bordes 0.941423
   Dwight Gibbs 0.941391
   Jason Keath  0.941360
   Helen Todd   0.941146
   scotwingo    0.941138
   Jason Kintzler 0.941130
   Kira Wampler 0.941104
   Ramit Sethi  0.941079
   Kathy Calvin 0.940925
   Rachel Levy  0.940899
   kelkelly     0.940862
   Jacquelyn Cyr 0.940862
   Brian Armstrong 0.940809
   Steve Goldstein 0.940762
   Michael Hyatt 0.940756
   Joel Dehlin  0.940750
   Kristina Halvorson 0.940730
   Jeremy Toeman 0.940725
   Cassandra Bailey 0.940701



In [172]:
cosine_value = []
Similar_CEO_Names = []
for c, w in reversed(c2[-30:]):
    cosine_value.append(c)
    Similar_CEO_Names.append(w)

cosine_value=pd.Series(cosine_value)
Similar_CEO_Names=pd.Series(Similar_CEO_Names)
list_similar_CEOs = pd.DataFrame()
list_similar_CEOs["cosine_value"] = cosine_value
list_similar_CEOs["Similar_CEO_Names"] = Similar_CEO_Names
list_similar_CEOs
user_name_list = []
#a = [frame["CEO_User_Name"] for x in list_similar_CEOs if x in frame]
#print(frame.columns)
for x in list_similar_CEOs["Similar_CEO_Names"]:
    for j in range(len(frame["CEO_Full_Name"])):
        if x == frame["CEO_Full_Name"][j]:
            if frame["CEO_User_Name\r"][j] not in user_name_list:
                user_name_list.append(frame["CEO_User_Name\r"][j])
user_name_list=pd.Series(user_name_list)
list_similar_CEOs["Similar_CEO_User_Names"] = user_name_list

In [186]:
list_similar_CEOs.to_csv("Similar_CEOs_Names_List.csv", sep = "\t",encoding='utf-8')

In [175]:
list_similar_CEOs

,cosine_value,Similar_CEO_Names,Similar_CEO_User_Names
0,0.970864,Lauren Cook,laurencook\r
1,0.964673,Don Tapscott,dtapscott\r
2,0.964533,Phil Libin,plibin\r
3,0.963710,Sundar Pichai,sundarpichai\r
4,0.963100,Tom Hood,tomhood\r


## Conducting Support Vector Machine Model

In [311]:
lenght = len(list_similar_CEOs["cosine_value"].values)
X_train = list_similar_CEOs["cosine_value"][:lenght//2]
X_test = list_similar_CEOs["cosine_value"][lenght//2:]
y_train = list_similar_CEOs["Similar_CEO_Names"][:lenght//2]
y_test = list_similar_CEOs["cosine_value"][lenght//2:]


In [312]:
import numpy as np
x1 = np.array(X_train.values.tolist())
x2 = np.array(X_test.values.tolist())
y1 = np.array(y_train.tolist())
y2 = np.array(y_test.tolist())

In [314]:
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
# train classifier
clf = SVC(probability=True, kernel='rbf')
clf.fit(x1.reshape(-1, 1), y1)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [316]:
predictions = clf.predict_proba(x2.reshape(-1, 1))
predictions[0]

array([0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667])